<div style="text-align: center;">
    <h1>Naive Bayes</h1>
</div>

# Importar Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score, classification_report
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import SMOTE 
from collections import Counter
import matplotlib.pyplot as plt

# Carregar e explorar o dataset

In [ ]:
df = pd.read_csv('C:/Users/wesle/machine-learning-tecnicas/Dataset/depression_anxiety_data.csv')

In [ ]:
# Visualizar as Primeiras Linhas
print(df.head(5))

In [ ]:
# Resumo Estatístico
print(df.describe())

In [ ]:
# Informações do DataFrame
print(df.info())

In [ ]:
# Valores Únicos por Coluna
print(df.nunique())

In [ ]:
# Verificar Dados Ausentes
print(df.isnull().sum())

In [ ]:
# Distribuição da Variável Alvo
plt.figure(figsize=(8, 6))
sns.countplot(x='depression_diagnosis', data=df)
plt.title('Distribuição da Variável Alvo')
plt.xlabel('Classes')
plt.ylabel('Contagem')
plt.show()

# Pré-processamento dos dados

In [ ]:
df.drop('id', axis = 1, inplace = True)

In [ ]:
print('Antes da remoção de missings:\n', df['depression_diagnosis'].value_counts())

In [ ]:
df.dropna(inplace = True) #removendo valores missing, 18 observações foram removidos (783-165)

In [ ]:
print('Depois da remoção de missings:\n', df['depression_diagnosis'].value_counts())

In [ ]:
df = df[df['who_bmi'] != 'Not Availble'] #a partir daqui, not availble não aparece mais, pega todos os elementos menos ele, isso significa que mais 8 observações foram removidas

In [ ]:
df.shape

In [ ]:
#gender
df['gender'] = df['gender'].map({'female':0,'male':1})

#who_bmi
df['who_bmi'] = df['who_bmi'].map( {
    "Underweight": 0,
    "Normal": 1,
    "Overweight": 2,
    "Class I Obesity": 3,
    "Class II Obesity": 4,
    "Class III Obesity": 5,
})

#depression_severity
df['depression_severity'] = df['depression_severity'].map({
    "none": 0,
    "None-minimal": 1,
    "Mild":2,
    "Moderate": 3,
    "Moderately severe": 4,
    "Severe": 5,
})

#anxiety_severity
df['anxiety_severity'] = df['anxiety_severity'].replace(0, 'none')

df['anxiety_severity'] = df['anxiety_severity'].replace({
    'none': 0,
    "None-minimal": 1,
    "Mild": 2,
    "Moderate": 3,
    "Severe": 4
})


In [ ]:
colunas = ['depressiveness','suicidal', 'depression_treatment', 'anxiousness', 'anxiety_diagnosis', 'anxiety_treatment', 'sleepiness', 'depression_diagnosis']

for coluna in colunas:
    df[coluna] = df[coluna].astype(int)

In [ ]:
df

# Separando as características (X) e rótulos (y)

In [ ]:
X = df.drop(columns=['depression_diagnosis'])
y = df['depression_diagnosis']

# Dividir o dataset em treino e teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Cenário 1: Dataset sem reamostragem (original)

## Treinamento do modelo

In [ ]:
naive_bayes = GaussianNB()

naive_bayes.fit(X_train, y_train)

## Avaliação do modelo

In [ ]:
y_pred = naive_bayes.predict(X_test)

#avaliando o modelo com o cross-validation
cv_scores = cross_val_score(naive_bayes, X_train, y_train, cv = 10, scoring='accuracy')

print('Relatório do modelo\n\n', classification_report(y_test, y_pred))
print(f"Acurácia média da validação cruzada: {cv_scores.mean() * 100:.2f}%")
print(f"Desvio padrão da acurácia: {cv_scores.std() * 100:.2f}%")

# Cenário 2: Dataset com reamostragem

## Balanceamento dos dados de treino

In [ ]:
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)
print(f"Distribuição das classes após SMOTE: {Counter(y_train_balanced)}")

## Treinamento do modelo

In [ ]:
naive_bayes.fit(X_train_balanced, y_train_balanced)

## Avaliação do modelo

In [ ]:
y_pred = naive_bayes.predict(X_test)

#avaliando o modelo com o cross-validation
cv_scores = cross_val_score(naive_bayes, X_train_balanced, y_train_balanced, cv = 10, scoring='accuracy')

print('Relatório do modelo\n\n', classification_report(y_test, y_pred))
print(f"Acurácia média da validação cruzada: {cv_scores.mean() * 100:.2f}%")
print(f"Desvio padrão da acurácia: {cv_scores.std() * 100:.2f}%")

# Cenário 3: Dataset com reamostragem e seleção de características

## Seleção de Características

In [ ]:
selector = SelectKBest(score_func = f_classif, k=5)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

view_features = X.columns[selector.get_support()]

print("Características selecionadas:\n", view_features)

## Balanceamento dos dados de treino selecionados

In [ ]:
X_train_balanced_selected, y_train_balanced_selected = smote.fit_resample(X_train_selected, y_train)

print(f"Distribuição das classes após SMOTE: {Counter(y_train_balanced_selected)}")

## Treinamento do modelo com dados reamostrados e selecionados

In [ ]:
naive_bayes.fit(X_train_balanced_selected, y_train_balanced_selected)

## Avaliação do Modelo

In [ ]:
y_pred_balanced_selected = naive_bayes.predict(X_test_selected)

#avaliando o modelo com o cross-validation
cv_scores = cross_val_score(naive_bayes, X_train_balanced_selected, y_train_balanced_selected, cv = 10, scoring='accuracy')

print('Relatório do modelo\n\n', classification_report(y_test, y_pred_balanced_selected ))
print(f"Acurácia média da validação cruzada: {cv_scores.mean() * 100:.2f}%")
print(f"Desvio padrão da acurácia: {cv_scores.std() * 100:.2f}%")